# CV – object detection

В этой тетрадке мы рассмотрим задачу детекции объектов на примере датасета [Stanford Drone Dataset](https://cvgl.stanford.edu/projects/uav_data/)

**Предполагаем, что ноутбук запущен внутри Yandex DataSphere**

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader, Sampler
from torchvision.models import resnet34
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms

import albumentations as A
import albumentations.pytorch.transforms as APT
from albumentations.pytorch import ToTensorV2

import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import datetime
from PIL import Image
from copy import deepcopy
from collections import defaultdict

import os

from skimage import io

## Data

Stanford drone dataset – это датасет из нескольких видео, записанных с дрона в восьми местах (`SCENE_NAME`).
Каждое видео покадрово размечено шестью типами объектов (`label`). (Подробное описание датасета лежит в файле `initial_README`)  
В текущей задаче мы будем использовать не исходные видео, а фреймы из них.

Структура директории с датасетом:
- ./stanford-drone-dataset-frames/
    - {train, val}/
        - annotations/
            - {`SCENE_NAME`}/
                - video{`VIDEO_ID`}/
                    - annotations.csv
    - {train, val}/
        - frames/
            - {`SCENE_NAME`}/
                - video{`VIDEO_ID`}/
                    - frame_{`FRAME_IDX`}.jpg

Для каждого файла с аннотацией есть аналогичный путь к директории, в которой находятся фреймы из видео `frame_{FRAME_IDX}.jpg`.  
Например, для фреймов, лежащих в директории `./stanford-drone-dataset-frames/train/frames/quad/video0/`, соответствующий файл с разметкой лежит по пути: `./stanford-drone-dataset-frames/train/annotations/quad/video0/annotations.csv`.

Каждая строка в файлах `annotations.csv` содержит аннотацию одного объекта. Каждый CSV файл содержит 11 колонок с хедером.

Наиболее интересные нам колонки:
- xmin – верхняя левая X-координата бокса объекта (bounding box).
- ymin – верхняя левая Y-координата бокса объекта (bounding box).
- xmax – нижняя правая X-координата бокса объекта (bounding box).
- ymax – нижняя правая Y-координата бокса объекта (bounding box).
- frame_idx – индекс фрейма, который соответсвует текущей строке аннотации.

Сэмпл из одной сцены можно скачать с [Google Drive](https://drive.google.com/file/d/18XeE0kHWqpLyBfZFbAbUTZqbygETTCLC/view?usp=sharing).

In [2]:
# # Качаем архив с данными с Yandex Object Storage.

# from cloud_ml.storage.api import Storage

# s3 = Storage.s3(access_key="Le9tg70HQEJsoGqjqXH8", secret_key="NV75mCPkC0PEd35ImyDI5vI7p40YGFOYZgkH7moa")
# # downloading contents of the remote file into the local one
# s3.get_dir('dl-hse-2021/stanford-drone-dataset-frames/', './stanford-drone-dataset-frames/')

## Задание 1
**(0.2 балла)** Напишите класс датасет, который будет возвращать картинку и координаты размеченных на ней объектов.

Обработка и преобразование датафреймов остались где-то за кадром. Если коротко говорить, то всю информацию объединил в 2 больших датафрейма. 

In [35]:
class StanfordDroneDataset(Dataset):
    def __init__(self, transform=None, csv='train.csv',
                 folder_path='./stanford-drone-dataset-frames/'):
        
        self.df = self.get_df(csv)
        self.folder_path = folder_path
        self.transform = transform
        
    def get_df(self, csv):
        df = pd.read_csv(csv)
        df = df[df.lost != 1].reset_index().drop('index', axis = 1)
        return df

    def __len__(self):
        return self.df.image_id.nunique()
    
    def __getitem__(self, index):
        image_id = torch.tensor([index])
        mini_df = self.df[self.df.image_id == index]
        try:
            image_path = mini_df.img_path.iloc[0]
            num_objs = mini_df.shape[0]
            # в датасете всего 2 лейбла
            mini_df['Enc_labels'] = mini_df.label.apply(lambda x: 0 if x == 'Pedestrian' else 1)

            image = io.imread(self.folder_path + image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = torch.as_tensor(image, dtype=torch.int32)

            #bboxes and labels
            boxes = []
            labels = []
            for i in range(num_objs):
                xmin = mini_df.xmin.iloc[i]
                xmax = mini_df.xmax.iloc[i]
                # тут какой-то косяк с bbox, подрезать будем так
                while xmax / image.shape[1] > 1:
                    xmax -= 1
                ymin = mini_df.ymin.iloc[i]
                ymax = mini_df.ymax.iloc[i]
                while ymax / image.shape[0] > 1:
                    ymax -= 1
                label = mini_df.label.iloc[i]
                boxes.append([xmin, ymin, xmax, ymax])
                labels.append(mini_df['Enc_labels'].iloc[i])
            boxes = torch.as_tensor(boxes, dtype=torch.int32)
            labels = torch.as_tensor(labels, dtype=torch.int32)

        
            target = {}
            target['image'] = image.numpy()
            target["bboxes"] = boxes.numpy()
            target["labels"] = labels.numpy()
            
        except:
            target = {}
            target['image'] = np.array([])
            target["bboxes"] = []
            target["labels"] = []
        

        if self.transform is not None:
            target = self.transform(**target)

        return target

In [4]:
CLASSES = ['Pedestrian', 'Biker']
train_data = StanfordDroneDataset(csv='./stanford-drone-dataset-frames/train.csv')
val_data = StanfordDroneDataset(csv='./stanford-drone-dataset-frames/train.csv')

Проверим и визуализурием работу датасета.

In [5]:
BOX_COLOR = (255, 0, 0) # Red
LINE_COLOR = (100, 100, 255)
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, label, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = [int(_) for _ in bbox]

    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)

    ((text_width, text_height), _) = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=label,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name, grid=None, centers=False, target_obj_mask=None):
    img = image.copy()
    if grid is not None:
        height, width, _ = img.shape
        nh, nw = grid
        
        for i in range(1, nw):
            x = (width // nw) * i
            cv2.line(img, (x, 0), (x, height), LINE_COLOR, 1)
        
        for j in range(1, nh):
            y = (height // nh) * j
            cv2.line(img, (0, y), (width, y), LINE_COLOR, 1)

    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [6]:
def show_examples(dataset, num_examples: int = 4):
    for i in range(num_examples):
        k = np.random.randint(0, 80000)
        target = dataset[k]
        visualize(target['image'], target['bboxes'], target['labels'], CLASSES, grid=(7, 7))
    
show_examples(dataset=train_data)

## Задание 2
(0.2 балла) сделайте подготовку данных на Albumentations, collate_fn и правильный даталоадер, проверьте шейпы выходных тензоров

#### Albumentations

На ShiftScaleRotate ловлю трейсбек, воспользуемся HorizontalFlip

In [7]:
target = train_data[2]

In [19]:
np.random.seed(2)
transform = A.Compose(
    [ 
        A.HorizontalFlip(p=0.5), 
        A.PadIfNeeded(min_height=512, min_width=512),
        A.RandomCrop(512, 512, always_apply=True),
        A.Normalize(),
        APT.ToTensorV2()
    ],
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels'],  min_visibility=.5),
)

transformed = transform(**target)

Проверка преобразований, но с тензорами не работает

In [20]:
# visualize(transformed['image'], transformed['bboxes'], transformed['labels'], CLASSES, grid=(7, 7))

#### collate_fn

In [11]:
def collate_fn(lst):
    tmp = defaultdict(list)
    ret = dict()
    for entry in lst:
        if entry['labels'] == []:
            continue
        for k, v in entry.items():
            tmp[k].append(v)
    
    for k, v in tmp.items():
        if isinstance(v[0], np.ndarray):
            v = np.concatenate(v, 0)
        if isinstance(v[0], torch.Tensor):
            v = torch.cat(v, 0)
        ret[k] = v
    return ret

In [36]:
trainset = StanfordDroneDataset(csv='./stanford-drone-dataset-frames/train.csv', transform=transform)  
valset = StanfordDroneDataset(csv='./stanford-drone-dataset-frames/val.csv', transform=transform)  

dl = DataLoader(trainset, shuffle=True, batch_size=32, collate_fn=collate_fn)

In [38]:
batch = next(iter(dl))

# --- пока так дальше доделаю, когда разберусь с Лоссом 

## Задание 3
(0.4 балла) Приготовьте модель для детекции, проверьте, что все правильно отрабатывает.

In [ ]:
class VeryModel(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x):
        pass
    
    def compute_all(self, batch):
        pass

net = VeryModel()
net.compute_all(batch)

## Задание 4
(0.2 балла) натренируйте модель:
- Убедитесь, что она учится,
- Проверьте, что на выходе что-то адекватное.

Трейнер можно взять с любого занятия.
